# L4: Measuring Search Relevance

In [1]:
import warnings
warnings.filterwarnings('ignore')

### WANDS Annotation Dataset
We need to download the <font color=red>**three csv files**</font> to the <font color=red>**data/**</font> folder of this project

https://github.com/wayfair/WANDS/tree/main/dataset  

It is a gold standard (possibly) annotation dataset  

The data is stored in the dataset folder in three files:  
  
1. **product.csv** - Stores all candidate products, columns include:  
a. product_id - ID of a product  
b. product_name - String of product name  
c. product_class - Category which product falls under  
d. category_hierarchy - Parent categories of product, delimited by /  
e. product_description - String description of product  
f. product_features - | delimited string of attribute:value pairs which describe the product   
g. rating_count - Number of user ratings for product  
h. average_rating - Average rating the product received  
i. review_count - Number of user reviews for product  
    
2. **query.csv** - Stores search queries, columns include:  
a. query_id - unique ID for each query  
b. query - query string  
c. query_class - category to which the query falls under  
  
3. **label.csv** - Stores annotated (product,relevance judgement) pairs, columns include  
a. id - Unique ID for each annotation    
b. query_id - ID of the query this annotation is for    
c. product_id - ID of the product this annotation applies to  
d. label - Relevance label, one of 'Exact', 'Partial', or 'Irrelevant'  

In [2]:
import pandas as pd

products_df = pd.read_csv(
    "data/wands/product.csv",
    sep="\t", # Tab separated columns
    index_col="product_id", # Consider it as an index
    keep_default_na=False,  # Some products do not have a description
)
products_df.head()

,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
product_id,,,,,,,,
0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0
3,all-clad all professional tools pizza cutter,"Slicers, Peelers And Graters",Browse By Brand / All-Clad,this original stainless tool was designed to c...,overallwidth-sidetoside:3.5|warrantylength : l...,69.0,4.5,42.0
4,baldwin prestige alcott passage knob with roun...,Door Knobs,Home Improvement / Doors & Door Hardware / Doo...,the hardware has a rich heritage of delivering...,compatibledoorthickness:1.375 '' |countryofori...,70.0,5.0,42.0


<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> For semantic search, we will choose <b>product_name</b> and <b>product_description</b> for better applicability

In [10]:
# We have large strings in product_description column
# This is the 6th string in the column (for demonstration)
print(len(products_df['product_description'][5]))

1642


In [11]:
num_products = 5000

In [12]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

product_name_embeddings = model.encode(products_df["product_name"][0:num_products].tolist())
product_name_embeddings.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(5000, 384)

In [13]:
product_description_embeddings = model.encode(products_df["product_description"][0:num_products].tolist())
product_description_embeddings.shape

(5000, 384)

### How `SentenceTransformer` Generates Embeddings per Sentence (RECAP)

When using `SentenceTransformer`, the model encodes each product name (or sentence) as a single vector, capturing the entire sentence’s meaning. Here’s a breakdown of the process using the examples `"Winter Jacket"` and `"Cozy Sweater"`.

#### Step-by-Step Process

1. **Tokenization**: The model splits each product name into tokens based on its vocabulary.
   - `"Winter Jacket"` → `["winter", "jacket"]`
   - `"Cozy Sweater"` → `["cozy", "sweater"]`
   
   Each word (or subword, in the case of uncommon words) is represented by tokens that are mapped to unique IDs corresponding to the model’s pretrained embeddings.

2. **Embedding Calculation**:
   - After tokenization, each token is represented by an embedding from the model's vocabulary.
   - The model combines these embeddings, typically by averaging or through weighted attention mechanisms, to produce a **single fixed-size vector** for each sentence.

#### Final Output
For each product name, a single vector represents the combined meaning of the sentence, reflecting the relationships between the tokens.

### Example:
- `"Winter Jacket"` results in **one embedding vector** representing the whole phrase.
- `"Cozy Sweater"` results in **one embedding vector** for the entire phrase.

The model does <font color=red>**not create separate embeddings**</font> for each word within a product name.


## Building the collection

In [17]:
products_df.index.shape

(42994,)

In [23]:
# Check the existence of collection
client.collection_exists(collection_name="wands-products")

True

In [28]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")
client.delete_collection("wands-products") # Delete any pre-existing collection named wands-products

True

In [29]:
client.collection_exists(collection_name="wands-products")

False

In [30]:
# Create the collection
client.create_collection(
    collection_name="wands-products",
    vectors_config={
        "product_name": models.VectorParams(size=384, distance=models.Distance.COSINE, ),
        "product_description": models.VectorParams(size=384, distance=models.Distance.COSINE,),
    },
    optimizers_config=models.OptimizersConfigDiff(default_segment_number=2, indexing_threshold=1000,),
)

True

In [44]:
client.collection_exists(collection_name="wands-products")

True

In [45]:
# Check the details of the collection
print(client.get_collection(collection_name="wands-products"))

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=9088 points_count=5000 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors={'product_description': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'product_name': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=2, max_segme

<p style="background-color:orange; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> The "<b>ids</b>=products_df.<b>index</b>.tolist()" part should insert entire 42k indices into the vector db, whereas, the embeddings have only 5000 entries <b>points count = 5000</b>. So, the part was not confirmed whether the excess indices cause any indexing issue in the db, although the functionalities work alright

In [41]:
client.upload_collection(
    collection_name="wands-products",
    vectors={
        "product_name": product_name_embeddings,
        "product_description": product_description_embeddings,
    },
    payload=products_df.to_dict(orient="records"), # Converting from an existing df
    ids=products_df.index.tolist(), # Index of the existing df
    batch_size=64, # The insert will be performed as batches of 64
)

In [46]:
client.count("wands-products") # Total count of the products

CountResult(count=5000)

In [43]:
# Check the details of the collection
print(client.get_collection(collection_name="wands-products"))

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=9088 points_count=5000 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors={'product_description': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'product_name': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=2, max_segme

In [47]:
import time

time.sleep(1.0)
collection = client.get_collection("wands-products")
while collection.status != models.CollectionStatus.GREEN: # Green = Vector DB Index ready
    time.sleep(1.0)
    collection = client.get_collection("wands-products")
    
collection # Now we access the entire set of Collections when everything is ready

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=9088, points_count=5000, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'product_description': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'product_name': VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segmen

## Test queries

<div>
<img src="img/l4_relevancy_1.png" width="400"/>
</div>

<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> Some relevancy based metrics : <br>
    <b><font color="blue">Precision</b> = Correct positive predictions out of all the <b>predicted positives</b> <br>
    <b><font color="blue">Recall</b> = Correct positive predictions out of all the <b>actual positives</b> <br>
    <b><font color="blue">MRR</b> = Average of the <b>reciprocal ranks</b> of the first relevant result across all queries <br>
    <b><font color="blue">NDCG</b> = Measures the ranking quality by giving higher-weight to relevant documents at <b>top ranks</b>

In [49]:
queries_df = pd.read_csv(
    "data/WANDS/query.csv", 
    sep="\t", 
    index_col="query_id",
)
queries_df.head()

,query,query_class
query_id,,
0,salon chair,Massage Chairs
1,smart coffee table,Coffee & Cocktail Tables
2,dinosaur,Kids Wall Décor
3,turquoise pillows,Accent Pillows
4,chair and a half recliner,Recliners


## Ground truth

In [50]:
labels_df = pd.read_csv(
    "data/WANDS/label.csv", 
    sep="\t", 
)
labels_df.sample(n=5)

,id,query_id,product_id,label
25454,25458,174,32869,Irrelevant
144722,145289,351,35496,Partial
108499,108651,215,17770,Partial
203703,204270,0,22573,Partial
25248,25252,171,38069,Partial


Now we have  
- `products_df` mapping all the products and their details
- `queries_df` mapping all the queries only
- `labels_df` mapping all ground truths, combining relevant `query-product-label`

In [59]:
# Printing the query_id 2
print(f'Query id 2 : {queries_df.loc[2]}')

# To see a particular query in multiple entries showing different level of matches
print(labels_df[(labels_df['query_id'] == 2) & (labels_df['label'] == 'Exact')])

Query id 2 : query                 dinosaur
query_class    Kids Wall Décor
Name: 2, dtype: object
          id  query_id  product_id  label
252      252         2        4205  Exact
253      253         2        4202  Exact
254      254         2        4204  Exact
255      255         2       36622  Exact
256      256         2       29777  Exact
...      ...       ...         ...    ...
406      406         2       42516  Exact
21668  21668         2       18868  Exact
21669  21669         2       24099  Exact
21670  21670         2       21055  Exact
21671  21671         2       23033  Exact

[151 rows x 4 columns]


In [60]:
relevancy_scores = {
    "Exact": 10,
    "Partial": 5,
    "Irrelevant": 0,
}

labels_df["score"] = labels_df["label"].map(relevancy_scores.get) # .map() applies the function relevancy_scores.get() on each row of the labels_df["label"]
labels_df["query_id"] = labels_df["query_id"].map(lambda x: f"query_{x}") # Takes the row value "x", then applies it with a prefix "query_" to make it query_x
labels_df["product_id"] = labels_df["product_id"].map(lambda x: f"doc_{x}")
labels_df.sample(n=5) # Shows a random sample of 5 rows

,id,query_id,product_id,label,score
20856,20856,query_141,doc_15654,Partial,5
57438,57589,query_454,doc_16769,Partial,5
176526,177093,query_426,doc_13573,Irrelevant,0
122793,122967,query_257,doc_24187,Partial,5
184846,185413,query_439,doc_23602,Irrelevant,0


## ranx

<p style="background-color:#edfbff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> The evaluation metrics operate on <b>Qrels = Query Relvancies</b> and <b>Runs = Output of the models</b> and then cross compare to give us the result of relevant metrics

In [61]:
from ranx import Qrels

# Create a Qrels (Query Relevance) dictionary from df 
qrels = Qrels.from_df(
    labels_df.astype({"query_id": "str", "product_id": "str"}),
    q_id_col="query_id",
    doc_id_col="product_id", 
    score_col="score",
)

ModuleNotFoundError: No module named 'ranx'

### Running all the queries

In [ ]:
# Encode every query and store them in a separate column in the queries_df
queries_df["query_embedding"] = model.encode(queries_df["query"].tolist()).tolist()
queries_df.sample(n=5)

In [ ]:
from collections import defaultdict

name_run_dict = defaultdict(dict)
for id, row in queries_df.iterrows():
    query_id = f"query_{id}"
    
    results = client.search(
        collection_name="wands-products",
        query_vector=models.NamedVector(
            name="product_name", 
            vector=row["query_embedding"]
        ),
        with_vectors=False,
        with_payload=False,
        limit=100,
    )

    for point in results:
        document_id = f"doc_{point.id}"
        name_run_dict[query_id][document_id] = point.score  
    
name_run_dict

In [ ]:
from ranx import Run

product_name_run = Run(name_run_dict, name="product_name")

In [ ]:
description_run_dict = defaultdict(dict)
for id, row in queries_df.iterrows():
    query_id = f"query_{id}"
    
    results = client.search(
        collection_name="wands-products",
        query_vector=models.NamedVector(
            name="product_description", 
            vector=row["query_embedding"]
        ),
        with_vectors=False,
        with_payload=False,
        limit=100,
    )

    for point in results:
        document_id = f"doc_{point.id}"
        description_run_dict[query_id][document_id] = point.score 

product_description_run = Run(
    description_run_dict, 
    name="product_description"
)

In [ ]:
from ranx import compare

compare(
    qrels=qrels,
    runs=[
        product_name_run, 
        product_description_run
    ],
    metrics=[
        "precision@10", 
        "recall@10", 
        "mrr@10",
        "dcg@10", 
        "ndcg@10",
    ],
)